In [1]:
import torch
import src
import wandb
import numpy as np
import os
import cv2
import random

from PIL import Image
from sklearn.model_selection import KFold

In [2]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [3]:
ENCODER = 'resnet152'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multicalss segmentation
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu') 
PARAMS = {
    "lr" : 0.0001,
    "batch_size" : 1,
    "num_splits" : 3,
}
set_random_seed(345262)

In [4]:
loss = src.utils.base.SumOfLosses(
    src.utils.losses.DiceLoss(),
    src.utils.losses.BCELoss()
)

metrics = [
    src.utils.metrics.IoU(threshold=0.5)
]

In [5]:
wandb.login()
wandb.init(project="Trus_images")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dalyutkin01. Use `wandb login --relogin` to force relogin


In [6]:
preprocess = src.utils.dataset.Preprocessing(
    dir='./data/',
    img_size=768,
    augmentation=False
)

In [7]:
clear_images, clear_masks = preprocess.load_folder()
kfold = KFold(n_splits=PARAMS["num_splits"], shuffle=True)
fold_data = []
for fold, (train_indices, test_indices) in enumerate(kfold.split(clear_masks)):
    train_data = {
        "images" : [clear_images[index] for index in train_indices],
        "masks" : [clear_masks[index] for index in train_indices]
    }
    test_data = {
        "images" : [clear_images[index] for index in test_indices],
        "masks" : [clear_masks[index] for index in test_indices]
    }
    fold_data.append((train_data, test_data))

In [8]:
# import matplotlib.pyplot as plt
# img = clear_images[1]
# # cv2.imshow("original", img)
 
# # Display cropped image
# b, g, r = cv2.split(img)
# np.multiply(b, 1.4, out=b, casting="unsafe")
# np.multiply(g, 1.7, out=g, casting="unsafe")
# np.multiply(r, 3, out=r, casting="unsafe")

# after = cv2.merge([b, g, r])

# cv2.imshow('before', img)
# cv2.imshow('after', after)
# cv2.waitKey()

In [9]:
# img = preprocess.prepocessing_image(clear_images[0])
# img.sum()
# cv2.imshow('after', img)
# cv2.waitKey()

In [10]:
track = 0
TEST_IMAGES = 0
TEST_MASKS = 0
loader_for_top_model = 0
for fold, (train_data, test_data) in enumerate(fold_data):
    print(f"Training on fold {fold + 1}...")
    model = src.FPN(
        encoder_name=ENCODER, 
        encoder_weights=ENCODER_WEIGHTS,
        in_channels=3,
        classes=1,
        activation=ACTIVATION,
    )
    optimizer = torch.optim.Adam([dict(params=model.parameters(), lr=PARAMS["lr"]),])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
        factor=0.1, patience=1, threshold=0.0001, threshold_mode='abs')
    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.02)
    train_epoch = src.utils.train.TrainEpoch(
        model, 
        loss=loss, 
        metrics=metrics, 
        optimizer=optimizer,
        device=DEVICE,
        verbose=True,
    )
    valid_epoch = src.utils.train.ValidEpoch(
        model,
        loss=loss, 
        metrics=metrics, 
        device=DEVICE,
        verbose=True,
    )
    
    train_dataloader, valid_dataloader = preprocess.Generator(
        batch_size=PARAMS["batch_size"],
        X_train=train_data["images"],
        y_train=train_data["masks"],
        X_test=test_data["images"],
        y_test=test_data["masks"]
    )

    for epoch in range(0, 15):
        print('\nEpoch: {}'.format(epoch + 1))
        train_logs = train_epoch.run(train_dataloader)
        valid_logs = valid_epoch.run(valid_dataloader)
        
        wandb.log({
            f'fold_{fold+1}/train/train_IoU': train_logs['iou_score'],
            f'fold_{fold+1}/train/train_loss': train_logs['dice_loss + bce_loss']
        })
        wandb.log({
            f'fold_{fold+1}/valid/valid_IoU': valid_logs['iou_score'], 
            f'fold_{fold+1}/valid/valid_loss': valid_logs['dice_loss + bce_loss']
        })
        
        scheduler.step(valid_logs['dice_loss + bce_loss'])
        if track < valid_logs['iou_score']:
            TEST_IMAGES = test_data["images"]
            TEST_MASKS = test_data["masks"]
            track = valid_logs['iou_score']
            loader_for_top_model = test_data
            torch.save({
                'model_state_dict': model.state_dict(),
                'best_IoU' : track
                }, f'./checkpoint/best_model_{PARAMS["num_splits"]}fold.pth')
            print('Model saved!')
wandb.finish()

Training on fold 1...

Epoch: 1
valid: 100%|██████████| 34/34 [00:13<00:00,  2.59it/s, dice_loss + bce_loss - 0.7677, iou_score - 0.2337] 
Model saved!

Epoch: 2
valid: 100%|██████████| 34/34 [00:11<00:00,  2.98it/s, dice_loss + bce_loss - 0.7646, iou_score - 0.2671]
Model saved!

Epoch: 3
valid: 100%|██████████| 34/34 [00:11<00:00,  3.00it/s, dice_loss + bce_loss - 0.7774, iou_score - 0.2919]
Model saved!

Epoch: 4
valid: 100%|██████████| 34/34 [00:11<00:00,  2.99it/s, dice_loss + bce_loss - 0.7929, iou_score - 0.2585] 

Epoch: 5
valid: 100%|██████████| 34/34 [00:11<00:00,  3.02it/s, dice_loss + bce_loss - 0.7395, iou_score - 0.307] 
Model saved!

Epoch: 6
valid: 100%|██████████| 34/34 [00:11<00:00,  3.01it/s, dice_loss + bce_loss - 0.7773, iou_score - 0.2804]

Epoch: 7
valid: 100%|██████████| 34/34 [00:11<00:00,  3.08it/s, dice_loss + bce_loss - 0.8312, iou_score - 0.2484]

Epoch: 8
valid: 100%|██████████| 34/34 [00:11<00:00,  3.04it/s, dice_loss + bce_loss - 0.7922, iou_score - 0.27

fold_1/train/train_IoU,▁▃▄▅▆▇▇████████
fold_1/train/train_loss,█▆▅▄▃▂▂▁▁▁▁▁▁▁▁
fold_1/valid/valid_IoU,▁▄▇▃█▅▂▅▇▆▇▅▆▅▆
fold_1/valid/valid_loss,▃▃▄▅▁▄█▅▃▄▃▅▅▅▄
fold_2/train/train_IoU,▁▃▄▅▆▇█████████
fold_2/train/train_loss,█▆▅▃▂▁▁▁▁▁▁▁▁▁▁
fold_2/valid/valid_IoU,▆▇▄▇▁▆█▅▆▇█▇▇▆▆
fold_2/valid/valid_loss,▃█▄▁▆▂▁▃▃▂▁▂▂▂▂
fold_3/train/train_IoU,▁▃▅▅▆▆▆▇███████
fold_3/train/train_loss,█▆▄▄▃▂▂▂▁▁▁▁▁▁▁
fold_3/valid/valid_IoU,▁█▇▇█▁▇▆▆▇▅▆▅█▆


In [11]:
valid_pred_folder = './valid_predict_kfold_v1/'
checkpoint = torch.load(f'./checkpoint/best_model_{PARAMS["num_splits"]}fold.pth')
model.load_state_dict(checkpoint['model_state_dict'])
checkpoint['best_IoU']

0.34370550995691695

In [12]:
shape = TEST_IMAGES[0].shape[:2]
preds = []
for image in TEST_IMAGES:
    image_prepoc = preprocess.inference_image(image)
    image_prepoc = torch.tensor(np.array([image_prepoc]), dtype = torch.float)
    image_prepoc = image_prepoc.to(DEVICE)
    with torch.no_grad():
        predict = model.forward(image_prepoc)
    predict_value = predict.cpu().detach().numpy()
    preds.append(predict_value)

In [13]:
for num in range(len(TEST_IMAGES)):
    true_mask = TEST_MASKS[num]
    greenBGR = src.utils.plotting.Visual.green_chanel(true_mask)
    concat_true = cv2.addWeighted(TEST_IMAGES[num], 0.4, TEST_MASKS[num], 0.9, 0)
    
    #resize true image and create 3rd channels
    cvtColor = cv2.cvtColor(TEST_IMAGES[num], cv2.COLOR_BGR2GRAY )
    resize_image = cv2.resize(cvtColor, (1280, 872), interpolation = cv2.INTER_AREA)
    resize_image = cv2.cvtColor(resize_image, cv2.COLOR_BGR2RGB)
    
    #preproc predict
    predict_mask = src.utils.plotting.Visual.preproc_predict(preds[num])
    blueBGR_predict = src.utils.plotting.Visual.blue_chanel_predict(predict_mask)
    greenBGR_predict = src.utils.plotting.Visual.green_chanel_predict(predict_mask)
    
    #calc IoU
    IoU = src.utils.plotting.Visual.calc_IoU(greenBGR, greenBGR_predict)
    
    #concat two masks
    bitwiseAnd = cv2.bitwise_or(greenBGR, blueBGR_predict)
    
    #concat true images with mask
    concat_predict = cv2.addWeighted(resize_image, 0.4, bitwiseAnd, 0.7, 0)
    concat_predict_resize = cv2.resize(concat_predict, (1280, 872), interpolation = cv2.INTER_AREA)
    
    #put text
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (570, 50)
    fontScale = 1
    color = (0, 255, 255)
    thickness = 2
    image = cv2.putText(concat_predict_resize, 'IoU ' + str(round(IoU, 3)), org, font, 
                   fontScale, color, thickness, cv2.LINE_AA)
    
    #save 
    cv2.imwrite(filename=os.path.join(valid_pred_folder + str(num) + "_true" + '.png'), img=concat_true)
    cv2.imwrite(filename=os.path.join(valid_pred_folder + str(num) + "_predict" + '.png'), img=concat_predict_resize)